In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Adrenocortical_Cancer"
cohort = "GSE49278"

# Input paths
in_trait_dir = "../../input/GEO/Adrenocortical_Cancer"
in_cohort_dir = "../../input/GEO/Adrenocortical_Cancer/GSE49278"

# Output paths
out_data_file = "../../output/preprocess/Adrenocortical_Cancer/GSE49278.csv"
out_gene_data_file = "../../output/preprocess/Adrenocortical_Cancer/gene_data/GSE49278.csv"
out_clinical_data_file = "../../output/preprocess/Adrenocortical_Cancer/clinical_data/GSE49278.csv"
json_path = "../../output/preprocess/Adrenocortical_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Expression profiling by array of 44 adrenocortical carcinomas"
!Series_summary	"Gene expression profiles of adrenocortical carcinomas were analyzed using Affymetrix Human Gene 2.0 ST Array to identify homogeneous molecular subgroups"
!Series_overall_design	"Gene expression profiles of 44 adrenocortical carcinomas were hybridized using Affymetrix Human Gene 2.0 ST Array"
Sample Characteristics Dictionary:
{0: ['age (years): 70', 'age (years): 26', 'age (years): 53', 'age (years): 73', 'age (years): 15', 'age (years): 51', 'age (years): 63', 'age (years): 29', 'age (years): 79', 'age (years): 45', 'age (years): 43', 'age (years): 41', 'age (years): 37', 'age (years): 81', 'age (years): 68', 'age (years): 42', 'age (years): 59', 'age (years): 39', 'age (years): 25', 'age (years): 36', 'age (years): 24', 'age (years): 49', 'age (years): 75', 'age (years): 48', 'age (years): 54', 'age (years): 28', 'age (years): 40', 'age (years): 44', 'age (years): 52

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# From the background information: "Expression profiling by array" and "Gene expression profiles...using Affymetrix Human Gene 2.0 ST Array"
# This indicates the dataset contains gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# Trait (Adrenocortical Cancer)
# From characteristics dictionary, key 2 contains 'cell type: Adrenocortical carcinoma'
# This is constant across all samples, but since it's the trait we're studying, we'll use it
trait_row = 2

# Age
# From characteristics dictionary, key 0 contains age data
age_row = 0

# Gender
# From characteristics dictionary, key 1 contains gender data
gender_row = 1

# 2.2 Data Type Conversion

def convert_trait(value):
    """Convert trait value to binary (1 for adrenocortical carcinoma, 0 for normal)"""
    if not value or ':' not in value:
        return None
    
    value = value.split(':', 1)[1].strip().lower()
    
    if 'adrenocortical carcinoma' in value:
        return 1
    else:
        return None  # We don't have controls in this dataset

def convert_age(value):
    """Convert age value to continuous"""
    if not value or ':' not in value:
        return None
    
    try:
        age_str = value.split(':', 1)[1].strip()
        return float(age_str)
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    """Convert gender value to binary (0 for female, 1 for male)"""
    if not value or ':' not in value:
        return None
    
    gender = value.split(':', 1)[1].strip().upper()
    
    if gender == 'F':
        return 0
    elif gender == 'M':
        return 1
    else:
        return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Initial filtering and save metadata
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Extract clinical features
    clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,  # This variable should be defined in previous steps
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the dataframe
    preview = preview_df(clinical_df)
    print("Clinical DataFrame Preview:")
    print(preview)
    
    # Save to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Clinical DataFrame Preview:
{'GSM1196511': [1.0, 70.0, 0.0], 'GSM1196512': [1.0, 26.0, 0.0], 'GSM1196513': [1.0, 53.0, 0.0], 'GSM1196514': [1.0, 73.0, 1.0], 'GSM1196515': [1.0, 15.0, 0.0], 'GSM1196516': [1.0, 51.0, 0.0], 'GSM1196517': [1.0, 63.0, 1.0], 'GSM1196518': [1.0, 26.0, 0.0], 'GSM1196519': [1.0, 29.0, 1.0], 'GSM1196520': [1.0, 79.0, 0.0], 'GSM1196521': [1.0, 45.0, 0.0], 'GSM1196522': [1.0, 43.0, 0.0], 'GSM1196523': [1.0, 53.0, 0.0], 'GSM1196524': [1.0, 45.0, 0.0], 'GSM1196525': [1.0, 41.0, 0.0], 'GSM1196526': [1.0, 37.0, 0.0], 'GSM1196527': [1.0, 81.0, 0.0], 'GSM1196528': [1.0, 68.0, 1.0], 'GSM1196529': [1.0, 42.0, 0.0], 'GSM1196530': [1.0, 59.0, 0.0], 'GSM1196531': [1.0, 39.0, 0.0], 'GSM1196532': [1.0, 25.0, 0.0], 'GSM1196533': [1.0, 41.0, 0.0], 'GSM1196534': [1.0, 36.0, 0.0], 'GSM1196535': [1.0, 24.0, 0.0], 'GSM1196536': [1.0, 49.0, 0.0], 'GSM1196537': [1.0, 75.0, 0.0], 'GSM1196538': [1.0, 37.0, 0.0], 'GSM1196539': [1.0, 26.0, 0.0], 'GSM1196540': [1.0, 48.0, 0.0], 'GSM1196541

### Step 3: Gene Data Extraction

In [4]:
# 1. First get the file paths again to access the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the get_genetic_data function from the library to get the gene_data from the matrix_file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers) for future observation
print("First 20 gene/probe identifiers:")
print(gene_data.index[:20])


First 20 gene/probe identifiers:
Index(['16650001', '16650003', '16650005', '16650007', '16650009', '16650011',
       '16650013', '16650015', '16650017', '16650019', '16650021', '16650023',
       '16650025', '16650027', '16650029', '16650031', '16650033', '16650035',
       '16650037', '16650041'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers to determine if they are human gene symbols
# These identifiers (e.g., '16650001') appear to be numeric probe IDs, not human gene symbols
# Human gene symbols typically have alphanumeric patterns like "BRCA1", "TP53", etc.
# These appear to be probeset IDs from a microarray platform that need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['16657436', '16657440', '16657445', '16657447', '16657450'], 'RANGE_STRAND': ['+', '+', '+', '+', '+'], 'RANGE_START': [12190.0, 29554.0, 69091.0, 160446.0, 317811.0], 'RANGE_END': [13639.0, 31109.0, 70008.0, 161525.0, 328581.0], 'total_probes': [25.0, 28.0, 8.0, 13.0, 36.0], 'GB_ACC': ['NR_046018', nan, nan, nan, 'NR_024368'], 'SPOT_ID': ['chr1:12190-13639', 'chr1:29554-31109', 'chr1:69091-70008', 'chr1:160446-161525', 'chr1:317811-328581'], 'RANGE_GB': ['NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10']}


### Step 6: Gene Identifier Mapping

In [7]:
# Since we're facing difficulties in mapping the probe IDs to gene symbols,
# we'll take a different approach for this platform (Affymetrix Human Gene 2.0 ST Array)

print("Original gene expression data shape:", gene_data.shape)

# For this Affymetrix platform, we could try to load a proper annotation,
# but as an alternative approach, we'll use the probe IDs directly as gene identifiers
# This is a valid fallback when proper mapping isn't available

# Clean and prepare the gene data
gene_data.index.name = 'Gene'  # Rename index to 'Gene' to maintain consistency with later steps

# Since we're using probe IDs as gene identifiers, no mapping is applied
# but we need to ensure the data is in the right format for subsequent steps

# Preview the gene data
print("\nUsing probe IDs as gene identifiers (first 5 rows):")
print(gene_data.head())

# Count the number of genes
print(f"\nNumber of genes/probes: {len(gene_data)}")

# Create the output directory if it doesn't exist
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)

# Save the gene data to a CSV file
gene_data.to_csv(out_gene_data_file)
print(f"Gene data saved to {out_gene_data_file}")

# Add a note about the gene mapping approach
print("\nNOTE: For this dataset, probe IDs are being used directly as gene identifiers")
print("because appropriate gene mapping information was not available in the provided annotation.")


Original gene expression data shape: (53617, 44)

Using probe IDs as gene identifiers (first 5 rows):
          GSM1196511  GSM1196512  GSM1196513  GSM1196514  GSM1196515  \
Gene                                                                   
16650001    3.114460    2.761934    3.191700    2.981038    3.113831   
16650003    2.070307    1.831540    2.303189    2.430376    1.507325   
16650005    2.532754    3.371765    2.264750    2.647668    2.559651   
16650007    1.968311    2.229541    1.762466    2.827752    1.626150   
16650009    1.418189    1.314710    1.571579    1.233351    1.753973   

          GSM1196516  GSM1196517  GSM1196518  GSM1196519  GSM1196520  ...  \
Gene                                                                  ...   
16650001    2.687413    3.468881    2.411585    3.761057    2.974074  ...   
16650003    2.382929    2.808405    2.031501    2.797925    2.567698  ...   
16650005    3.508271    1.959297    2.764491    2.655120    1.712738  ...   
16650007

Gene data saved to ../../output/preprocess/Adrenocortical_Cancer/gene_data/GSE49278.csv

NOTE: For this dataset, probe IDs are being used directly as gene identifiers
because appropriate gene mapping information was not available in the provided annotation.


### Step 7: Data Normalization and Linking

In [8]:
# 1. Since normalization of gene symbols failed (no IDs recognized), let's use the original gene data
print("Loading original gene expression data...")
gene_data = get_genetic_data(matrix_file)

# Save the gene data with original probe IDs
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")
print(f"Gene data shape: {gene_data.shape}")

# 2. Load previously saved clinical data and link with genetic data
clinical_data_path = out_clinical_data_file
clinical_data = pd.read_csv(clinical_data_path)
print(f"Clinical data columns: {clinical_data.columns.tolist()}")

# The clinical data CSV likely has columns named by sample IDs from transpose
# We need to set the columns as index before linking
clinical_data = clinical_data.set_index(clinical_data.columns[0])

# Transpose clinical data to have samples as rows and features as columns
clinical_data = clinical_data.T
print(f"Transposed clinical data shape: {clinical_data.shape}")

# Rename the columns to standard names
if len(clinical_data.columns) >= 3:
    clinical_data.columns = [trait, 'Age', 'Gender']
elif len(clinical_data.columns) == 2:
    clinical_data.columns = [trait, 'Age']
elif len(clinical_data.columns) == 1:
    clinical_data.columns = [trait]

# Transpose gene data to have samples as rows and genes as columns
gene_data_t = gene_data.T

# Merge clinical and genetic data
linked_data = pd.concat([clinical_data, gene_data_t], axis=1)
print(f"Linked data shape: {linked_data.shape}")

# Check for the trait column
print(f"Linked data columns (first 5): {linked_data.columns[:5].tolist()}")

# 3. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)
print(f"After handling missing values, linked data shape: {linked_data.shape}")

# 4. Determine whether the trait and demographic features are severely biased
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
print(f"Is trait biased: {is_biased}")

# 5. Conduct final quality validation and save cohort information
note = "Dataset containing gene expression profiles of adrenocortical carcinomas. All samples are cancer cases (no controls), making the trait binary variable biased."
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note=note
)

# 6. If the linked data is usable, save it
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for trait-gene association studies.")

Loading original gene expression data...


Gene expression data saved to ../../output/preprocess/Adrenocortical_Cancer/gene_data/GSE49278.csv
Gene data shape: (53617, 44)
Clinical data columns: ['GSM1196511', 'GSM1196512', 'GSM1196513', 'GSM1196514', 'GSM1196515', 'GSM1196516', 'GSM1196517', 'GSM1196518', 'GSM1196519', 'GSM1196520', 'GSM1196521', 'GSM1196522', 'GSM1196523', 'GSM1196524', 'GSM1196525', 'GSM1196526', 'GSM1196527', 'GSM1196528', 'GSM1196529', 'GSM1196530', 'GSM1196531', 'GSM1196532', 'GSM1196533', 'GSM1196534', 'GSM1196535', 'GSM1196536', 'GSM1196537', 'GSM1196538', 'GSM1196539', 'GSM1196540', 'GSM1196541', 'GSM1196542', 'GSM1196543', 'GSM1196544', 'GSM1196545', 'GSM1196546', 'GSM1196547', 'GSM1196548', 'GSM1196549', 'GSM1196550', 'GSM1196551', 'GSM1196552', 'GSM1196553', 'GSM1196554']
Transposed clinical data shape: (43, 3)
Linked data shape: (44, 53620)
Linked data columns (first 5): ['Adrenocortical_Cancer', 'Age', 'Gender', '16650001', '16650003']


After handling missing values, linked data shape: (43, 53620)
Quartiles for 'Adrenocortical_Cancer':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1.0
Max: 1.0
The distribution of the feature 'Adrenocortical_Cancer' in this dataset is severely biased.

Quartiles for 'Age':
  25%: 29.5
  50% (Median): 43.0
  75%: 53.0
Min: 15.0
Max: 81.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 8 occurrences. This represents 18.60% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

Is trait biased: True
Dataset is not usable for trait-gene association studies.
